# Working on Basic Algorithm Fitting

The goal is to start testing some basic framework, first with a sample made-up set of inputs and data, then generalize to our official set locally, and finally work on integration with front end and perhaps distributed approach.

In [25]:
%matplotlib inline

import numpy as np
import scipy as sp
import numpy as np
import pandas as pd
import random as rd
import itertools
import re
import matplotlib.pyplot as plt
from scipy.optimize import linprog

### Create Inputs


In [2]:
# basic macro requirement based on https://www.bodybuilding.com/fun/macronutrients_calculator.htm
# given in grams
# assume 5 day work week

daily_macro_nutrients = {'carb': 433, 'protein': 289, 'fat': 107}
weekly_macro_nutrients = dict([(x, daily_macro_nutrients[x]*5) for x in daily_macro_nutrients.keys()])

recipes = {}

for i in range(25):
    key_name = 'recipe'+str(i)
    rd.seed(i)
    macros = rd.sample(range(30), 3)
    recipes[key_name] ={'carb': macros[0], 'protein': macros[1], 'fat': macros[2]}

In [3]:
weekly_macro_nutrients

{'carb': 2165, 'fat': 535, 'protein': 1445}

In [4]:
recipes

{'recipe0': {'carb': 25, 'fat': 12, 'protein': 22},
 'recipe1': {'carb': 4, 'fat': 22, 'protein': 25},
 'recipe10': {'carb': 17, 'fat': 6, 'protein': 12},
 'recipe11': {'carb': 13, 'fat': 27, 'protein': 16},
 'recipe12': {'carb': 14, 'fat': 4, 'protein': 19},
 'recipe13': {'carb': 7, 'fat': 25, 'protein': 20},
 'recipe14': {'carb': 3, 'fat': 19, 'protein': 21},
 'recipe15': {'carb': 28, 'fat': 22, 'protein': 0},
 'recipe16': {'carb': 10, 'fat': 12, 'protein': 14},
 'recipe17': {'carb': 15, 'fat': 28, 'protein': 24},
 'recipe18': {'carb': 5, 'fat': 10, 'protein': 19},
 'recipe19': {'carb': 20, 'fat': 15, 'protein': 23},
 'recipe2': {'carb': 28, 'fat': 2, 'protein': 1},
 'recipe20': {'carb': 27, 'fat': 22, 'protein': 20},
 'recipe21': {'carb': 4, 'fat': 19, 'protein': 20},
 'recipe22': {'carb': 28, 'fat': 0, 'protein': 4},
 'recipe23': {'carb': 27, 'fat': 26, 'protein': 28},
 'recipe24': {'carb': 21, 'fat': 5, 'protein': 25},
 'recipe3': {'carb': 7, 'fat': 11, 'protein': 16},
 'recipe4':

Now consider a simple loss function on similarity.  Greedy search to minimize total difference in target nutrients, no preference to what is omitted to start.  Simplifying assumption that cannot have partial units of recipe.

In [5]:
current_nutrients = {'carb': 0, 'protein': 0, 'fat': 0}
total_loss = weekly_macro_nutrients

Simulate the removal of certain recipes due to ingredient lacking. 

In [6]:
rd.seed(45)
removal_indices = rd.sample(range(25), 13)
available_recipes = {}
for key in recipes.keys():
    numerical_id = re.match(r"([a-z]+)([0-9]+)", key, re.I).groups()[1]
    
    if int(numerical_id) not in removal_indices:
        available_recipes[key] = recipes[key]

In [7]:
available_recipes

{'recipe10': {'carb': 17, 'fat': 6, 'protein': 12},
 'recipe12': {'carb': 14, 'fat': 4, 'protein': 19},
 'recipe13': {'carb': 7, 'fat': 25, 'protein': 20},
 'recipe14': {'carb': 3, 'fat': 19, 'protein': 21},
 'recipe15': {'carb': 28, 'fat': 22, 'protein': 0},
 'recipe16': {'carb': 10, 'fat': 12, 'protein': 14},
 'recipe17': {'carb': 15, 'fat': 28, 'protein': 24},
 'recipe21': {'carb': 4, 'fat': 19, 'protein': 20},
 'recipe24': {'carb': 21, 'fat': 5, 'protein': 25},
 'recipe3': {'carb': 7, 'fat': 11, 'protein': 16},
 'recipe4': {'carb': 7, 'fat': 11, 'protein': 3},
 'recipe9': {'carb': 13, 'fat': 4, 'protein': 11}}

Think there is some assumption of minimum number of recipes to ensure we dont just end up with one recipe that generically scales and matches all macro count.  

In [8]:
def optimize_with_single_n(possible_list, target_nutrs):
    recipe_combination = list(itertools.combinations(possible_list, 1))
    cleaned_combination = [x[0] for x in recipe_combination]
    
    best_loss = {'carb': 1000, 'protein': 1000, 'fat': 1000}
    for iteration in cleaned_combination:
        
        recipe = possible_list[iteration]
        current_carb, current_protein, current_fat = 0, 0, 0
        
        next_carb = current_carb + recipe['carb']
        next_protein = current_protein + recipe['protein']
        next_fat = current_fat + recipe['fat']
        
        if next_carb > target_nutrs['carb'] or next_protein > target_nutrs['protein'] or next_fat > target_nutrs['fat']:
            print 'This recipe does not fit the target nutrients'
        
        count = 1
        while next_carb <= target_nutrs['carb'] and next_protein <= target_nutrs['protein'] and next_fat <= target_nutrs['fat']:
            current_carb = next_carb
            current_protein = next_protein
            current_fat = next_fat
            
            next_carb = current_carb + recipe['carb']
            next_protein = current_protein + recipe['protein']
            next_fat = current_fat + recipe['fat']
            count += 1
    
    
        carb_loss = str(target_nutrs['carb'] - current_carb)
        protein_loss = str(target_nutrs['protein'] - current_protein)
        fat_loss = str(target_nutrs['fat'] - current_fat)
        
        
        print 'For '+str(count)+" servings of "+iteration+" carb is "+str(current_carb)+", protein is "+str(current_protein)+", fat is "+str(current_fat)+"."
        print 'Loss is '+carb_loss+' carbs, '+protein_loss+' protein, '+fat_loss+' fat.'
        print '\n'
            

In [9]:
optimize_with_single_n(available_recipes, weekly_macro_nutrients)

For 132 servings of recipe9 carb is 1703, protein is 1441, fat is 524.
Loss is 462 carbs, 4 protein, 11 fat.


For 49 servings of recipe3 carb is 336, protein is 768, fat is 528.
Loss is 1829 carbs, 677 protein, 7 fat.


For 49 servings of recipe4 carb is 336, protein is 144, fat is 528.
Loss is 1829 carbs, 1301 protein, 7 fat.


For 25 servings of recipe15 carb is 672, protein is 0, fat is 528.
Loss is 1493 carbs, 1445 protein, 7 fat.


For 29 servings of recipe14 carb is 84, protein is 588, fat is 532.
Loss is 2081 carbs, 857 protein, 3 fat.


For 20 servings of recipe17 carb is 285, protein is 456, fat is 532.
Loss is 1880 carbs, 989 protein, 3 fat.


For 45 servings of recipe16 carb is 440, protein is 616, fat is 528.
Loss is 1725 carbs, 829 protein, 7 fat.


For 90 servings of recipe10 carb is 1513, protein is 1068, fat is 534.
Loss is 652 carbs, 377 protein, 1 fat.


For 22 servings of recipe13 carb is 147, protein is 420, fat is 525.
Loss is 2018 carbs, 1025 protein, 10 fat.


F

Next attempt to generalize to passing a set number of recipes and finding optimal combination of each set to minimize distance to target nutrients.  The thought is that this can then be easily extended to input of number main dishes, sides, etc.  We still assume an equality of nutrient in that there is no preference between what is minimized.  Later this could be changed based on diet preference, i.e. if I select Keto, I could have a preference band to allow 100g of extra fat and protien in order to minimize carbs.  By recognizing that without a preference of loss, we can simplify objective function to be total nutrient miss and pass individual macro constraint.

In [100]:
def optimize_with_multiple_n(possible_list, target_nutrs, number_recipes, minimum = 0):
    
    recipe_combination = list(itertools.combinations(possible_list, number_recipes))
    cleaned_combination = recipe_combination
    
    counter = 1
    best_loss = sum(target_nutrs.values())
    best_recipes = None
    best_weights = None
    
    for iteration in cleaned_combination:
        
        print "Optimizing combination "+str(counter)
        print "\n"
        macro_vector = [(possible_list[x]['carb'], possible_list[x]['protein'], possible_list[x]['fat']) for x in iteration]
        
        # define objective function as total macro loss with minimization
        c = [sum(target_nutrs.values())]+[-sum(x) for x in macro_vector]
        
        # define constraints, add 0 to account for constant
        carb_const = [0]+[x[0] for x in macro_vector]
        protien_const = [0]+[x[1] for x in macro_vector]
        fat_const = [0]+[x[2] for x in macro_vector]
        A = [carb_const, protien_const, fat_const]
        
        # define inequalities
        b = target_nutrs.values()
        
        # define bounds, note x0 must be 1 to define minimization
        # might want minimum of each
        x0_bounds = (1, 1)
        var_bounds = [x0_bounds]+[(minimum, None) for x in range(len(iteration))]

        
        #run optimization
        res = linprog(c, A_ub=A, b_ub=b, bounds=var_bounds, options={"disp": False})
        
        if res['fun'] < best_loss:
            
            print 'The new best loss is '+str(round(res['fun'], 4))
            print '\n'
            best_loss = res['fun']
        
        best_recipes = iteration
        best_weights = res['x'][1:]
        counter = counter + 1
        
    return best_recipes, best_weights

In [101]:
recipes, weights = optimize_with_multiple_n(available_recipes, weekly_macro_nutrients, 2)

Optimizing combination 1


The new best loss is 466.8182


Optimizing combination 2


The new best loss is 453.945


Optimizing combination 3


The new best loss is 445.124


Optimizing combination 4


Optimizing combination 5


Optimizing combination 6


Optimizing combination 7


The new best loss is 440.8333


Optimizing combination 8


Optimizing combination 9


Optimizing combination 10


Optimizing combination 11


Optimizing combination 12


Optimizing combination 13


Optimizing combination 14


Optimizing combination 15


Optimizing combination 16


Optimizing combination 17


Optimizing combination 18


Optimizing combination 19


Optimizing combination 20


Optimizing combination 21


Optimizing combination 22


Optimizing combination 23


Optimizing combination 24


Optimizing combination 25


Optimizing combination 26


Optimizing combination 27


Optimizing combination 28


Optimizing combination 29


Optimizing combination 30


Optimizing combination 31


Optimizing comb

In [102]:
recipes, weights

(('recipe24', 'recipe21'), array([ 44.68,  16.4 ]))

In [115]:
recipes, weights = optimize_with_multiple_n(available_recipes, weekly_macro_nutrients, 3)

Optimizing combination 1


The new best loss is 453.945


Optimizing combination 2


The new best loss is 445.124


Optimizing combination 3


Optimizing combination 4


Optimizing combination 5


Optimizing combination 6


The new best loss is 440.8333


Optimizing combination 7


Optimizing combination 8


Optimizing combination 9


Optimizing combination 10


Optimizing combination 11


Optimizing combination 12


Optimizing combination 13


Optimizing combination 14


Optimizing combination 15


Optimizing combination 16


Optimizing combination 17


Optimizing combination 18


Optimizing combination 19


Optimizing combination 20


Optimizing combination 21


Optimizing combination 22


Optimizing combination 23


Optimizing combination 24


Optimizing combination 25


Optimizing combination 26


Optimizing combination 27


Optimizing combination 28


Optimizing combination 29


Optimizing combination 30


Optimizing combination 31


Optimizing combination 32


Optimizing combinat

In [116]:
recipes, weights

(('recipe12', 'recipe24', 'recipe21'), array([  0.  ,  44.68,  16.4 ]))